<a href="https://colab.research.google.com/github/deeplearning-itba/cnn/blob/master/Trabajo_Final_CNN_Style_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Style Transfer

<img src="https://i0.wp.com/chelseatroy.com/wp-content/uploads/2018/12/neural_style_transfer.png?resize=768%2C311&ssl=1">

La idea de este trabajo final es reproducir el siguiente paper:

https://arxiv.org/pdf/1508.06576.pdf

El objetivo es transferir el estilo de una imagen dada a otra imagen distinta. 

Como hemos visto en clase, las primeras capas de una red convolucional se activan ante la presencia de ciertos patrones vinculados a detalles muy pequeños.

A medida que avanzamos en las distintas capas de una red neuronal convolucional, los filtros se van activando a medida que detectan patrones de formas cada vez mas complejos.

Lo que propone este paper es asignarle a la activación de las primeras capas de una red neuronal convolucional (por ejemplo VGG19) la definición del estilo y a la activación de las últimas capas de la red neuronal convolucional, la definición del contenido.

La idea de este paper es, a partir de dos imágenes (una que aporte el estilo y otra que aporte el contenido) analizar cómo es la activación de las primeras capas para la imagen que aporta el estilo y cómo es la activación de las últimas capas de la red convolucional para la imagen que aporta el contenido. A partir de esto se intentará sintetizar una imagen que active los filtros de las primeras capas que se activaron con la imagen que aporta el estilo y los filtros de las últimas capas que se activaron con la imagen que aporta el contenido.

A este procedimiento se lo denomina neural style transfer.

# En este trabajo se deberá leer el paper mencionado y en base a ello, entender la implementación que se muestra a continuación y contestar preguntas sobre la misma.

# Una metodología posible es hacer una lectura rápida del paper (aunque esto signifique no entender algunos detalles del mismo) y luego ir analizando el código y respondiendo las preguntas. A medida que se planteen las preguntas, volviendo a leer secciones específicas del paper terminará de entender los detalles que pudieran haber quedado pendientes.

Lo primero que haremos es cargar dos imágenes, una que aporte el estilo y otra que aporte el contenido. A tal fin utilizaremos imágenes disponibles en la web.

In [1]:
# Imagen para estilo
!wget https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg

# Imagen para contenido
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg

# Creamos el directorio para los archivos de salida
!mkdir /content/output

'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
The syntax of the command is incorrect.


In [4]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse
from keras.applications import vgg19
from keras import backend as K
from pathlib import Path
import tensorflow as tf
#tf.compat.v1.disable_eager_execution()

In [25]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida

base_image_path = Path("C:/Users/arile/iCloudDrive/Itba/Deep Learning/Diplomatura/Trabajo Final/Style Transfer/content/Neckarfront_Tübingen_Mai_2017.jpg")
style_reference_image_path = Path("/Users/arile/iCloudDrive/Itba/Deep Learning/Diplomatura/Trabajo Final/Style Transfer/content/La_noche_estrellada1.jpg")
result_prefix = Path("/Users/arile/iCloudDrive/Itba/Deep Learning/Diplomatura/Trabajo Final/Style Transfer/content/output")
iterations = 100

In [6]:
#base_image_path = Path("C:/Users/arile/iCloudDrive/Itba/Deep Learning/Diplomatura/Trabajo Final/Style Transfer/content/Big-buddah.jpg")
#style_reference_image_path =  Path("/Users/arile/iCloudDrive/Itba/Deep Learning/Diplomatura/Trabajo Final/Style Transfer/content/Composition_VIII.jpg")

# 1) En base a lo visto en el paper ¿Qué significan los parámetros definidos en la siguiente celda?

Respuesta:

- Total_variation_weight: Es el valor por el cual se pesa la total_variation_loss, la cual actúa como un término regulador

- Style_weight: Es el valor por el cual se pesa la style_loss. En el paper se representa con $\alpha$

- Content_weight: Es el valor por el cual se pesa la content_loss. En el paper se representa con $\beta$


In [26]:
total_variation_weight = 0.1
style_weight = 10
content_weight = 1

In [27]:
# Definimos el tamaño de las imágenes a utilizar
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

# 2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué?

Ayuda: https://keras.io/applications/

Respuesta:
- Primera línea: Carga la imagen en la variable img, utilizando el path provisto y le asigna las medidas img_nrows para la altura y img_ncols para el ancho.
- Segunda línea: Convierte la imagen en un array
- Tercera línea: Agrega una nueva dimensión al array de la imagen. Esta dimensión cumple la función del batch size.
    - Luego de esta línea, el shape de la variable img será (batch size, alto, ancho, canales).
- Cuarte línea: Aplica a la imagen el preprocesamiento requerido para los inputs, por la VGG19 en Keras.
- Ultima línea: Devuelve la imagen pre procesada

In [28]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# 3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

Respuesta:

En la primera línea de Código, se le saca al array X (la imagen), la dimensión agregada con el expand_dims en la función anterior.
Luego, se procede a revertir todos los cambios que realiza "vgg19.preprocess_input" en la imagen.

En resumen, lo que obtengo como salida de la función es idéntico a lo que obtengo luego de utilizar las 2 primeras líneas de código de la función preprocess_image


In [29]:
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [30]:
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para 
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

In [31]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

Aclaración:

La siguiente celda sirve para procesar las tres imagenes (contenido, estilo y salida) en un solo batch.

In [32]:
# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [33]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

Model loaded.


# 4) En la siguientes celdas:

- ¿Qué es la matriz de Gram? ¿Para qué se usa?
    - La matriz de Gram es la multiplicación de una matriz con su matriz transpuesta, donde el objetivo es que cada valor interno sea igual al cuadrado del original
    - Se utiliza para calcular la independencia lineal
- ¿Por qué se permutan las dimensiones de x?
    - Para poder realizar la matriz de Gram, necesito que las dimensiones de mi matriz sean (Cantidad de canales, (Alto * Ancho))
    - La función K.batch_flatten conserva la primera dimensión y aplica un “flatten” entre las restantes.
         - Por lo tanto, necesito que quede posicionada primera mi dimensión de la cantidad de canales y por último mis dimensiones de alto y ancho de la imagen.

In [34]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# 5) Losses:

Explicar qué mide cada una de las losses en las siguientes tres celdas.

Rta:

- Style loss: Minimiza la distancia media cuadrada entre la matriz de Gram de la imagen original y la matriz de Gram de la imagen generada
    - Esto lo hace evaluando las matrices en cada una de las capas definidas más adelante en la lista "feature_layers"
- Content loss: Minimiza la distancia media cuadrada entre los “features” de la imagen generada menos los features de la imagen original.
- Total variation loss: Mide la cantidad de sonido que posee la imagen.
    - Se utiliza para reducir el sonido que posee la imagen
    - Actúa como un término de regularización


In [35]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [36]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

In [37]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [38]:
# Armamos la loss total
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                            combination_features)
feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']

for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :] 
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [39]:
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

# 6) Explique el propósito de las siguientes tres celdas. ¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

Respuesta:



- Función eval_loss_and_grads:
    - Primero se hace un reshape de la entrada x
        - x = x.reshape((1, img_nrows, img_ncols, 3))
     - Se utiliza la K.function definida en la celda anterior, la cual devuelve en primera posición la loss y en la segunda, los gradientes
         - outs = f_outputs([x])
     - Se guarda el valor de la loss obtenida al usar la función en el punto anterior.
         - loss_value = outs[0]
     - Dependiendo del largo de outs, se hace el flatten sobre la salida de los gradientes de la función f_outputs()
         - if len(outs[1:]) == 1:
             - grad_values = outs[1].flatten().astype('float64')
         - else:
             - grad_values = np.array(outs[1:]).flatten().astype('float64')
     - Se devuelve el valor de la loss y los gradientes
         - return loss_value, grad_values

- Clase Evaluator:
    - Permite calcular la loss y el gradiente en una sola pasada y las guarda en 2 funciones separadas.
    - Esto se hace ya que scipy.optimize necesita funciones separadas para la loss y el gradiente.
- Función Evaluator.loss
    - Primero se asegura de que el valor self.loss.value, sea nulo
    - Luego, utiliza la función eval_loss_and_grads para calcular el valor de la loss y el gradiente.
    - Por último, los almacena en self.loss_value y self.grads_values y devuelve loss_value
- Función Evaluator.grads
    - Primero comprueba que self.loss_value, no sea nulo
    - Luego, copia los valores de self.grad_values a grad_values
    - Por último, limpia los valores self.loss_value y self.grad_values y devuelve grad_values

- Tercera celda:
    - Primero se instancia la clase Evaluator() y se trae la imagen de base y se la pre procesa.
        - evaluator = Evaluator()
        - x = preprocess_image(base_image_path)
    - Luego se realiza un for de n iteraciones, donde cada una realiza lo siguiente:
        - Muestra la iteración en la cual se encuentra
            - print('Start of iteration', i)
        - Se toma el tiempo inicial
            - start_time = time.time()
        - Se calcula la loss utilizando fmin_l_bfgs_b
            - x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(), fprime=evaluator.grads, maxfun=20)
        - Se muestra el valor de la loss
            - print('Current loss value:', min_val)
        - Se desprocesa la imagen de la salida para poder visualizarla luego
            - img = deprocess_image(x.copy())
        - Se guarda la imagen
            - fname = result_prefix / ('output_at_iteration_%d.png' % i)
            - save_img(fname, img)
        - Se toma el tiempo final
            - end_time = time.time()
        - Se imprime el nombre del archivo guardado y el tiempo que se tomo para realizar la iteración
            - print('Image saved as', fname)
            - print('Iteration %d completed in %ds' % (i, end_time - start_time))

- Función fmin_l_bfgs_b:
    - Minimiza una función dada, utilizando el algoritmo L-BFGS-B
        - Algoritmo BFGS:
            - Es un método iterativo, que permite resolver problemas de optimización no lineales, determinando la dirección de descenso del gradiente.
            - Para calcularlo, utiliza la inversa de la matriz Hessiana y guarda una matriz densa de n x n, donde n es la cantidad de variables en el problema
        - Algoritmo L-BFGS (Limited Memory BFGS)
            - A diferencia de BFGS, L-BFGS utiliza para su matriz solo algunos vectores que representan una aproximación implícita. Para realizar esto, L-BFGS conserva historia de las ultimas m posiciones del gradiente de x.
        - Algoritmo L-BFGS-B
            - Sumado a lo destacado en los puntos anteriores, permite utilizar restricciones en las variables en formato de limites superiores e inferiores.
    - Dentro del Código, se utilizaron los siguientes valores para la función
        - Función para minimizar --> Evaluator.loss
        - Punto de partida / valor inicial --> preprocess_image(imagen_base).flatten() (x.flatten())
        - Gradiente de la función a minimizar --> fprime=evaluator.grads
        - Cantidad máxima de evaluaciones de la funcion --> maxfun=20
    - Esta funcion, devuelve 3 valores.
        - En la primera posición, devuelve un array con la posición estimada del mínimo, para este caso, la imagen modificada.
        - En la segunda posición, devuelve el valor de la funcion en dicho mínimo.
        - En la tercera posición, devuelve un diccionario informativo con
            - información si convergió, o si no lo hico debido a que pasaron muchas iteraciones u otra razón.
            - Si el gradiente se encuentra en el mínimo
            - La cantidad de llamados a la funcion realizados
            - Cantidad de iteraciones realizadas
            
- Diferencia entre el paper y la notebook.
    - La principal diferencia que se puede observar entre la implementación de esta notebook y el paper, es el uso de la total_variation_loss
        - En el paper, utilizan únicamente la relación entre el content_weight y el style_weight, para regular el ruido que se percibe en la imagen y el nivel de "combinación deseado"
    - Otra diferencia que se puede apreciar es que, en el paper, deciden utilizar average pooling en vez de max pooling, ya que con esto obtienen mejores resultados.

In [40]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.

In [41]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

# 7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

*Ver imágenes adjuntas (Pesos-default.rar)

In [106]:
#total_variation_weight = 0.1
#style_weight = 10
#content_weight = 1

In [19]:
evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('default/output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 13207581000.0
Image saved as \Users\arile\iCloudDrive\Itba\Deep Learning\Diplomatura\Trabajo Final\Style Transfer\content\output\default\output_at_iteration_0.png
Iteration 0 completed in 7s
Start of iteration 1
Current loss value: 6206455000.0
Image saved as \Users\arile\iCloudDrive\Itba\Deep Learning\Diplomatura\Trabajo Final\Style Transfer\content\output\default\output_at_iteration_1.png
Iteration 1 completed in 5s
Start of iteration 2
Current loss value: 4238181000.0
Image saved as \Users\arile\iCloudDrive\Itba\Deep Learning\Diplomatura\Trabajo Final\Style Transfer\content\output\default\output_at_iteration_2.png
Iteration 2 completed in 5s
Start of iteration 3
Current loss value: 3352174600.0
Image saved as \Users\arile\iCloudDrive\Itba\Deep Learning\Diplomatura\Trabajo Final\Style Transfer\content\output\default\output_at_iteration_3.png
Iteration 3 completed in 5s
Start of iteration 4
Current loss value: 2703199200.0
Image saved as \Users

# 8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)

Respuesta:

En el primer ejemplo, decidí generar nuevas imágenes modificando únicamente la style_weight, la cual paso de 10 a 10.000, dejando el valor en para el ratio entre α y β en 10^-4. 
La diferencia se puede distinguir fácilmente en la imagen 99 de ambas salidas. En las imágenes, se puede apreciar, que aquellas generadas utilizando un style_weight de 10.000, poseen una mayor deformación de la imagen original, con mayores curvaturas y colores propios de la imagen de estilo utilizada.

El segundo ejemplo, opte por dejar la style weight en su valor original y modificar la content weight, la cual fue modificada de 1 a 100. Esto nos deja con un ratio entre el α y β de 10. En este caso, como era de esperarse, se puede distinguir rápidamente que los colores de la foto son mucho más similares a los de la imagen base y el estilo de la noche estrellada, se encuentra mucho menos presente.


In [24]:
#total_variation_weight = 0.1
#style_weight = 10000
#content_weight = 1

In [42]:
evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('pesos-modificados-2/output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 33830273000.0
Image saved as \Users\arile\iCloudDrive\Itba\Deep Learning\Diplomatura\Trabajo Final\Style Transfer\content\output\pesos-modificados-2\output_at_iteration_0.png
Iteration 0 completed in 6s
Start of iteration 1
Current loss value: 21282253000.0
Image saved as \Users\arile\iCloudDrive\Itba\Deep Learning\Diplomatura\Trabajo Final\Style Transfer\content\output\pesos-modificados-2\output_at_iteration_1.png
Iteration 1 completed in 5s
Start of iteration 2
Current loss value: 17152663000.0
Image saved as \Users\arile\iCloudDrive\Itba\Deep Learning\Diplomatura\Trabajo Final\Style Transfer\content\output\pesos-modificados-2\output_at_iteration_2.png
Iteration 2 completed in 5s
Start of iteration 3
Current loss value: 14927238000.0
Image saved as \Users\arile\iCloudDrive\Itba\Deep Learning\Diplomatura\Trabajo Final\Style Transfer\content\output\pesos-modificados-2\output_at_iteration_3.png
Iteration 3 completed in 5s
Start of iteration 4
Cur

# 9) Cambiar las imágenes de contenido y estilo por unas elegidas por usted. Adjuntar el resultado.

Respuesta:

*Ver imágenes adjuntas (Estilo-e-imagen-propio.rar)

Para esta punto, decidí utilizar varias combinaciones distintas.
Decidí utilizar imágenes de base con bastante simples y otras un poco más complejas, así como también para el caso de las imágenes de estilo.
En cuanto a los pesos de las losses, dejé los que estaban en la notebook por default.

In [ ]:
#total_variation_weight = 0.1
#style_weight = 100
#content_weight = 1

In [23]:
evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('fotos-nuevas/Budah-composition/output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 44890880000.0
Image saved as \Users\arile\iCloudDrive\Itba\Deep Learning\Diplomatura\Trabajo Final\Style Transfer\content\output\fotos-nuevas\Budah-composition\output_at_iteration_0.png
Iteration 0 completed in 5s
Start of iteration 1
Current loss value: 23769098000.0
Image saved as \Users\arile\iCloudDrive\Itba\Deep Learning\Diplomatura\Trabajo Final\Style Transfer\content\output\fotos-nuevas\Budah-composition\output_at_iteration_1.png
Iteration 1 completed in 3s
Start of iteration 2
Current loss value: 17235649000.0
Image saved as \Users\arile\iCloudDrive\Itba\Deep Learning\Diplomatura\Trabajo Final\Style Transfer\content\output\fotos-nuevas\Budah-composition\output_at_iteration_2.png
Iteration 2 completed in 3s
Start of iteration 3
Current loss value: 14072140000.0
Image saved as \Users\arile\iCloudDrive\Itba\Deep Learning\Diplomatura\Trabajo Final\Style Transfer\content\output\fotos-nuevas\Budah-composition\output_at_iteration_3.png
Iteratio